In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from shapely.geometry import Point
import matplotlib.pyplot as plt
import geopandas as gpd 
from geopandas import GeoDataFrame
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabanairline/tabanAirline.csv
/kaggle/input/airports/airports.csv


In [3]:
import plotly.express as px

In [4]:
#read airports.csv
dfAirports=pd.read_csv("/kaggle/input/airports/airports.csv")
dfAirports.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
0,6523,00A,heliport,Total Rf Heliport,40.070801,-74.933601,11.0,NaN,US,US-PA,Bensalem,no,00A,NaN,00A,NaN,NaN,NaN
1,323361,00AA,small_airport,Aero B Ranch Airport,38.704022,-101.473911,3435.0,NaN,US,US-KS,Leoti,no,00AA,NaN,00AA,NaN,NaN,NaN
2,6524,00AK,small_airport,Lowell Field,59.949200,-151.695999,450.0,NaN,US,US-AK,Anchor Point,no,00AK,NaN,00AK,NaN,NaN,NaN
3,6525,00AL,small_airport,Epps Airpark,34.864799,-86.770302,820.0,NaN,US,US-AL,Harvest,no,00AL,NaN,00AL,NaN,NaN,NaN
4,6526,00AR,closed,Newport Hospital & Clinic Heliport,35.608700,-91.254898,237.0,NaN,US,US-AR,Newport,no,NaN,NaN,NaN,NaN,NaN,00AR


In [5]:
#Extracting airport information in Iran
irDfAirport=dfAirports.loc[dfAirports['iso_country'] =='IR']

In [7]:
#Selecting a subset of columns in table irDfAirport
cdfirDfAirport=irDfAirport[['ident','name','latitude_deg','longitude_deg','elevation_ft','continent','iso_region','gps_code']]

In [8]:
cdfirDfAirport.head()

,ident,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_region,gps_code
25840,IR-0001,Khaneh Airport,36.733341,45.150002,4804.0,AS,IR-02,OITH
25841,IR-0002,Ahmadi Military Air Field,29.099199,51.035301,NaN,AS,IR-06,NaN
25842,IR-0003,Darrahi Military Air Field,29.375799,51.067501,NaN,AS,IR-06,NaN
25843,IR-0004,Shahrokhi Highway Strip,35.232089,48.613558,NaN,AS,IR-24,NaN
25844,IR-0005,Shahabad Highway Strip,34.070489,46.597095,NaN,AS,IR-17,NaN


In [9]:
cdfirDfAirport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136 entries, 25840 to 53929
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ident          136 non-null    object 
 1   name           136 non-null    object 
 2   latitude_deg   136 non-null    float64
 3   longitude_deg  136 non-null    float64
 4   elevation_ft   113 non-null    float64
 5   continent      136 non-null    object 
 6   iso_region     136 non-null    object 
 7   gps_code       118 non-null    object 
dtypes: float64(3), object(5)
memory usage: 9.6+ KB


In [10]:
cdfirDfAirport.describe()

,latitude_deg,longitude_deg,elevation_ft
count,136.000000,136.000000,113.000000
mean,32.738760,52.332986,2815.035398
std,3.664746,3.962630,2195.662103
min,25.443300,44.430000,-79.000000
25%,29.547975,49.691600,101.000000
50%,32.931702,51.779150,3358.000000
75%,35.945754,54.465824,4404.000000
max,39.603600,62.315800,7385.000000


In [11]:
#check missing values
cdfirDfAirport.isna().sum()

ident             0
name              0
latitude_deg      0
longitude_deg     0
elevation_ft     23
continent         0
iso_region        0
gps_code         18
dtype: int64

In [13]:
#read tabanairline.csv for 1 to 13 farvardin
dfTabanAirline=pd.read_csv("/kaggle/input/tabanairline/tabanAirline.csv")
dfTabanAirline.head()

,flight_id,origin,destination,register,flight_num,airplane,airline_icao,dow,status_,status1_,status2_,type_,delay_,international,scheduled_date,scheduled_time,actual_date,actual_time,miladi_scheduled,miladi_actual
0,1,OIII,OIMM,EPTBF,TBN6258,MD82,TBN,سه شنبه,پرواز كرد,NaN,NaN,departure,22,0,1402-01-01,23:15:00,1402-01-01,23:37:00,21/3/2023,21/3/2023
1,2,OIMM,OIII,EPSMA,TBN6251,MD.88,TBN,سه شنبه,نشست,NaN,پایان دریافت بار,arrival,16,0,1402-01-01,22:40:00,1402-01-01,22:56:00,21/3/2023,21/3/2023
2,3,OIMM,OIII,EPTBF,TBN6257,MD82,TBN,سه شنبه,نشست,NaN,پایان دریافت بار,arrival,-5,0,1402-01-01,22:15:00,1402-01-01,22:10:00,21/3/2023,21/3/2023
3,4,OIMM,OIII,NaN,TBN6251,MD.88,TBN,سه شنبه,پرواز كرد,NaN,NaN,departure,37,0,1402-01-01,21:15:00,1402-01-01,21:52:00,21/3/2023,21/3/2023
4,5,ORNI,OIMM,NaN,TBN7266,A320,TBN,سه شنبه,نشست,NaN,پایان دریافت بار,arrival,-2,1,1402-01-01,21:10:00,1402-01-01,21:08:00,21/3/2023,21/3/2023


In [14]:
#rename 'inent' column for merge with tabanairline
cdfirDfAirport.rename(columns = {'ident':'origin'}, inplace = False)



,origin,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_region,gps_code
25840,IR-0001,Khaneh Airport,36.733341,45.150002,4804.0,AS,IR-02,OITH
25841,IR-0002,Ahmadi Military Air Field,29.099199,51.035301,NaN,AS,IR-06,NaN
25842,IR-0003,Darrahi Military Air Field,29.375799,51.067501,NaN,AS,IR-06,NaN
25843,IR-0004,Shahrokhi Highway Strip,35.232089,48.613558,NaN,AS,IR-24,NaN
25844,IR-0005,Shahabad Highway Strip,34.070489,46.597095,NaN,AS,IR-17,NaN
...,...,...,...,...,...,...,...,...
40131,OIZH,Zahedan International Airport,29.475700,60.906200,4564.0,AS,IR-13,OIZH
40132,OIZI,Iran Shahr Airport,27.236099,60.720001,2040.0,AS,IR-13,OIZI
40133,OIZJ,Jask Airport,25.653601,57.799301,19.0,AS,IR-23,OIZJ
40134,OIZS,Saravan Airport,27.419300,62.315800,3930.0,AS,IR-13,OIZS


In [16]:
#merge tabanairline with airlines
dfMrg=dfTabanAirline.merge(cdfirDfAirport.rename(columns = {'ident':'origin'}, inplace = False))

In [24]:
dfMrg.head()

,flight_id,origin,destination,register,flight_num,airplane,airline_icao,dow,status_,status1_,...,actual_time,miladi_scheduled,miladi_actual,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_region,gps_code
0,1,OIII,OIMM,EPTBF,TBN6258,MD82,TBN,سه شنبه,پرواز كرد,NaN,...,23:37:00,21/3/2023,21/3/2023,Mehrabad International Airport,35.689201,51.3134,3962.0,AS,IR-07,OIII
1,7,OIII,OIMM,NaN,TBN6252,MD.88,TBN,سه شنبه,نشست,NaN,...,20:44:00,21/3/2023,21/3/2023,Mehrabad International Airport,35.689201,51.3134,3962.0,AS,IR-07,OIII
2,8,OIII,OIMM,EPSMA,TBN6252,MD.88,TBN,سه شنبه,پرواز كرد,NaN,...,19:31:00,21/3/2023,21/3/2023,Mehrabad International Airport,35.689201,51.3134,3962.0,AS,IR-07,OIII
3,12,OIII,OIKQ,EPSMA,TBN6227,md88,TBN,سه شنبه,پرواز كرد,NaN,...,12:37:00,21/3/2023,21/3/2023,Mehrabad International Airport,35.689201,51.3134,3962.0,AS,IR-07,OIII
4,15,OIII,OIMM,NaN,TBN6224,MD.88,TBN,سه شنبه,نشست,NaN,...,09:13:00,21/3/2023,21/3/2023,Mehrabad International Airport,35.689201,51.3134,3962.0,AS,IR-07,OIII


In [44]:
#Obtaining the number of flight types based on airports
typeCount = dfMrg.groupby(['origin', 'type_']).size().reset_index(name='count')
typeCount

,origin,type_,count
0,OIAW,departure,1
1,OIBB,arrival,5
2,OIBK,arrival,20
3,OIFM,arrival,9
4,OIFM,departure,13
5,OIII,arrival,24
6,OIII,departure,66
7,OIKQ,arrival,5
8,OIMM,arrival,54
9,OIMM,departure,66


In [56]:
pivotedTypeCount = typeCount.pivot(index='origin', columns='type_', values='count').fillna(0).reset_index().fillna(0)
pivotedTypeCount

type_,origin,arrival,departure
0,OIAW,0.0,1.0
1,OIBB,5.0,0.0
2,OIBK,20.0,0.0
3,OIFM,9.0,13.0
4,OIII,24.0,66.0
5,OIKQ,5.0,0.0
6,OIMM,54.0,66.0
7,OISS,5.0,7.0
8,OIYY,1.0,0.0
9,OIZH,1.0,1.0


In [66]:
#add 'latitude_deg','longitude_deg','name' to above table
result=pivotedTypeCount.merge(cdfirDfAirport[['ident','latitude_deg','longitude_deg','name']].rename(columns = {'ident':'origin'}, inplace = False))
result

,origin,arrival,departure,total,latitude_deg,longitude_deg,name
0,OIAW,0.0,1.0,1.0,31.337400,48.762001,Ahwaz Airport
1,OIBB,5.0,0.0,5.0,28.944799,50.834599,Bushehr Airport
2,OIBK,20.0,0.0,20.0,26.526199,53.980202,Kish International Airport
3,OIFM,9.0,13.0,22.0,32.750801,51.861301,Esfahan Shahid Beheshti International Airport
4,OIII,24.0,66.0,90.0,35.689201,51.313400,Mehrabad International Airport
5,OIKQ,5.0,0.0,5.0,26.754601,55.902401,Dayrestan Airport
6,OIMM,54.0,66.0,120.0,36.235199,59.640999,Mashhad International Airport
7,OISS,5.0,7.0,12.0,29.539200,52.589802,Shiraz Shahid Dastghaib International Airport
8,OIYY,1.0,0.0,1.0,31.904900,54.276501,Shahid Sadooghi Airport
9,OIZH,1.0,1.0,2.0,29.475700,60.906200,Zahedan International Airport


In [69]:
# plot

fig = px.scatter_geo(result, lat='latitude_deg', lon='longitude_deg',
                     hover_name='name',
                     hover_data={'arrival': True, 'departure': True ,'latitude_deg':False , 'longitude_deg':False},scope='asia')
fig.show()

In [58]:
# fig = px.scatter_geo(pivotedTypeCount,lat='latitude_deg',lon='longitude_deg',scope='asia',width=700,height=800, hover_name='origin')
# fig.update_layout(title = 'Airports', title_x=0.5)
# fig.show()